# Energy Island Wind Farm Cluster

[Try this yourself](https://colab.research.google.com/github/DTUWindEnergy/TopFarm2/blob/master/docs/notebooks/wind_farm_cluster.ipynb) (requires google account)

In [ ]:
# Install TopFarm if needed
import importlib
if not importlib.util.find_spec("topfarm"):
    !pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/TopFarm2.git

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Polygon as PLTPolygon
from shapely.geometry import Polygon
from topfarm.examples.energy_island import EnergyIsland

In [ ]:
TFC = EnergyIsland()
TFC.x_target = TFC.x_target
TFC.y_target = TFC.y_target

In [ ]:
RPs = np.arange(10, 16).astype(int)
n_wt_list = (1000/RPs).astype(int)
wt_types = [5, 2, 4, 3, 1, 3, 2, 2, 2, 2]
n_wts = n_wt_list[wt_types].tolist()
random_pct = 50
seeds_ss = 10 * [0]
ws_ss = [10]
wd_ss = np.arange(0, 360, 30)
fn_prefix_ss = 'ss_states/ss_state'
fn_prefix_sim = 'sim/sim_res'
construction_days = [   0,  360,  720, 1080, 1440, 1800, 2160, 2520, 2880, 3240]
df = TFC.run(wt_types,
        n_wts,
        construction_days,
        seeds_ss,)


In [ ]:
df

In [ ]:
colors = list(mcolors.TABLEAU_COLORS)
fig, ax = plt.subplots(figsize=(8, 10))
for n, farm in enumerate(TFC.wind_farm_boundaries):
    polygon = Polygon(np.asarray(farm).T)
    if n==0:
        label = 'target farm'
    else:
        label = f'farm {n-1}'
        ax.text(polygon.centroid.x, polygon.centroid.y, f'{n-1}', color = colors[n],
               bbox={'facecolor':'white','alpha':1,'edgecolor':'none','pad':1},
              ha='center', va='center') 
    ax.plot(farm[0] + [farm[0][0]], farm[1] + [farm[1][0]], color=colors[n], label = label)
    ax.add_patch(PLTPolygon(np.asarray(farm).T, closed=True, color=colors[n]))
ax.set_aspect('equal', adjustable='box')
ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)

In [ ]:
xs = np.arange(3653)
plt.figure(figsize=(15,10))
plt.plot(xs, df.power_no_neighbours_no_wake.rolling(365).mean()/10**6, label='no wake loss')
plt.plot(xs, df.power_no_neighbours.rolling(365).mean()/10**6, label='with internal wake loss')
plt.plot(xs, df.power_no_wake.rolling(365).mean()/10**6, label='with external wake loss')
plt.plot(xs, df.power.rolling(365).mean()/10**6, label='with internal and external wake loss')
plt.legend()
plt.title('Monthly mean farm power for different loss types')
plt.xlabel('Month no.')
plt.ylabel('Farm power [MW]')
plt.savefig('monthly_mean_power.png')